In [27]:
import pandas as pd
import numpy as np
import duckdb

from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

In [28]:
# Caminho do banco
db_path = "../../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Carrega os dados da camada bronze
df = con.execute("""
    SELECT 
          CG.*
        , CC.cluster
                 
    FROM gold.consumo_geral AS CG
    
    INNER JOIN output.clusterizacao_cliente AS CC ON
        CG.client_id = CC.client_id
""").df()

# Feature Engineering

In [29]:
def gerar_lag_features(df: pd.DataFrame, coluna_alvo: str, num_lags: int) -> pd.DataFrame:
    """
    Gera lag features para a coluna alvo em um DataFrame.

    Parâmetros:
    df (pd.DataFrame): DataFrame original.
    coluna_alvo (str): Nome da coluna alvo para gerar os lags.
    num_lags (int): Quantidade de lags a serem geradas (ex: 12 gera lag_1 até lag_12).

    Retorna:
    pd.DataFrame: DataFrame com as novas colunas de lag adicionadas.
    """
    df_copy = df.copy()
    for lag in range(1, num_lags + 1):
        df_copy[f'{coluna_alvo}_lag_{lag}'] = df_copy[coluna_alvo].shift(lag)
    return df_copy

def criar_features_climaticas(df):
    """
    Cria colunas de amplitude térmica, faixa de temperatura e faixa de umidade em um DataFrame.

    Espera que o DataFrame tenha colunas:
        - temperature_max
        - temperature_min
        - temperature
        - humidity
    """
    df = df.copy()

    df['temperature_max'] = df['temperature'].max()
    df['temperature_min'] = df['temperature'].min()

    # Amplitude térmica
    df['amplitude_termica'] = df['temperature_max'] - df['temperature_min']

    # Faixa de temperatura média
    def classificar_temp(temp):
        if temp < 24.356250:
            return 'baixa'
        elif temp <= 25.813333:
            return 'media'
        else:
            return 'alta'

    df['faixa_temperatura'] = df['temperature'].apply(classificar_temp)

    # Faixa de umidade
    def classificar_umidade(h):
        if h < 58.623529:
            return 'baixa'
        elif h <= 61.540625:
            return 'media'
        else:
            return 'alta'

    df['faixa_umidade'] = df['humidity'].apply(classificar_umidade)

    return df

def criar_features_clientes(df):
    """
    Espera DataFrame com colunas: client_id, date (datetime) e consumption_kwh.
    Retorna um novo DataFrame com client_id, variabilidade_consumo_cliente e tendencia_consumo_cliente.
    """
    df = df.copy()
    df['date_ordinal'] = df['date'].apply(lambda x: x.toordinal())  # Facilita a regressão linear
    
    resultados = []

    for client, grupo in df.groupby('client_id'):
        # Variabilidade do consumo
        variabilidade = grupo['consumption_kwh'].std()

        # Tendência (regressão linear)
        if len(grupo) >= 2:
            X = grupo[['date_ordinal']]
            y = grupo['consumption_kwh']
            modelo = LinearRegression().fit(X, y)
            tendencia = modelo.coef_[0]  # Inclinação da reta
        else:
            tendencia = np.nan

        resultados.append({
            'client_id': client,
            'variabilidade_consumo_cliente': variabilidade,
            'tendencia_consumo_cliente': tendencia
        })

    return pd.DataFrame(resultados)


In [30]:
df = criar_features_climaticas(df)

# 1. Geração das features por cliente
df_features_clientes = criar_features_clientes(df)

# 2. Merge com o dataframe original de consumo
df = df.merge(
    df_features_clientes,
    on='client_id',
    how='left'
)

In [31]:
faixa_temperatura = [f'temperatura_{temp}' for temp in df['faixa_temperatura'].unique().tolist()]

# Codificação da variável categórica
df = pd.get_dummies(df, columns=['faixa_temperatura'], prefix='temperatura', prefix_sep='_')

df[faixa_temperatura] = df[faixa_temperatura].astype(int)

faixa_umidade = [f'umidade_{temp}' for temp in df['faixa_umidade'].unique().tolist()]

# Codificação da variável categórica
df = pd.get_dummies(df, columns=['faixa_umidade'], prefix='umidade', prefix_sep='_')

df[faixa_umidade] = df[faixa_umidade].astype(int)

In [32]:
df = gerar_lag_features(df, 'temperature', 14)
df = gerar_lag_features(df, 'humidity', 14)
df = gerar_lag_features(df, 'consumption_kwh', 14)

df = df.fillna(0)

# Treinamento de Modelo

In [33]:


def calcular_metricas(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    nrmse = rmse / np.mean(y_true)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, nrmse, r2

def treinar_modelos_xgb_por_grupo_cv(df, features):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    resultados = []

    grupos = df.groupby(['cluster', 'region'])

    for (cluster, region), grupo_df in grupos:
        print(f"\nTreinando modelo para Cluster {cluster} - Região {region}")

        X = grupo_df[features]
        y = grupo_df['consumption_kwh']
        tscv = TimeSeriesSplit(n_splits=5)

        maes, rmses, nrmses, r2s = [], [], [], []

        for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = xgb.XGBRegressor(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=3,
                random_state=42,
                n_jobs=-1
            )

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)

            mae, rmse, nrmse, r2 = calcular_metricas(y_val, y_pred)
            maes.append(mae)
            rmses.append(rmse)
            nrmses.append(nrmse)
            r2s.append(r2)

        # Salvar o modelo final com todos os dados
        modelo_final = xgb.XGBRegressor(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=3,
            random_state=42,
            n_jobs=-1
        )
        modelo_final.fit(X, y)

        resultados.append({
            'cluster': cluster,
            'region': region,
            'mae': np.mean(maes),
            'rmse': np.mean(rmses),
            'nrmse': np.mean(nrmses),
            'r2': np.mean(r2s)
        })

    return pd.DataFrame(resultados)


In [34]:
# Excluir colunas que não são features
features = [col for col in df.columns if col not in ['client_id', 'date', 'consumption_kwh', 'cluster', 'region']]

# Treinar modelos
resultados = treinar_modelos_xgb_por_grupo_cv(df, features)

# Exibir resultados
print(resultados.sort_values(by='nrmse'))



Treinando modelo para Cluster 0 - Região Leste

Treinando modelo para Cluster 0 - Região Norte

Treinando modelo para Cluster 1 - Região Centro

Treinando modelo para Cluster 1 - Região Oeste

Treinando modelo para Cluster 2 - Região Leste

Treinando modelo para Cluster 2 - Região Norte

Treinando modelo para Cluster 3 - Região Sul

Treinando modelo para Cluster 4 - Região Centro

Treinando modelo para Cluster 4 - Região Oeste
  cluster  region       mae      rmse     nrmse        r2
4       2   Leste  1.859354  2.308701  0.127443  0.121813
3       1   Oeste  1.876553  2.332099  0.128987 -0.045969
2       1  Centro  1.925851  2.377105  0.136699  0.007390
5       2   Norte  1.929484  2.408688  0.141294  0.152002
6       3     Sul  1.930882  2.425541  0.165105  0.571389
7       4  Centro  1.859306  2.342471  0.179367  0.243962
8       4   Oeste  1.901535  2.411506  0.183186  0.273257
0       0   Leste  1.887039  2.350982  0.199489  0.275713
1       0   Norte  1.858637  2.321256  0.19950